In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torch.optim as optim
from datetime import datetime, timedelta
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# ======================================================= DEFINICJA SIECI ==========================================================

class LeNet5CIFAR10(nn.Module):
    def __init__(self, num_layers, layer_sizes, dropout_rate):
        super(LeNet5CIFAR10, self).__init__()

        # Validate input
        if num_layers < 2 or num_layers > 10:
            raise ValueError("num_layers must be between 2 and 10")
        if len(layer_sizes) != num_layers:
            raise ValueError("layer_sizes length must match num_layers")
        
        self.conv1=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(dropout_rate)
        )

        self.conv2=nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(dropout_rate)
        )

        # Fully connected layers
        self.fcs = nn.ModuleList()  # Initialize as a list to hold the layers dynamically

        # Define the first fully connected layer, input size matches flattened conv output
        input_size = 16 * 5 * 5
        self.fcs.append(nn.Linear(input_size, layer_sizes[0]))

        # Define the intermediate layers based on `num_layers` and `layer_sizes`
        for i in range(1, num_layers - 1):
            self.fcs.append(nn.Linear(layer_sizes[i - 1], layer_sizes[i]))

        # Output layer
        self.fcs.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.reshape(out.size(0), -1)

        # Apply fully connected layers with ReLU activations for all but last
        for layer in self.fcs[:-1]:
            out = F.relu(layer(out))

        # Final layer (no activation to allow for softmax)
        # x = self.fcs[-1](x)
        return self.fcs[-1](out)

        # Apply softmax for class probabilities
        # return F.softmax(x, dim=1)

        print("END FORWARD")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
import datetime
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Podział danych treningowych na zbiór treningowy i walidacyjny (90% na trening, 10% na walidację)
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])


def train_and_eval(learning_rate, num_layers, layer_sizes, batch_size, dropout_rate, weight_decay):
  # Stworzenie DataLoaderów
  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
  val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

  net = LeNet5CIFAR10(num_layers, layer_sizes, dropout_rate).to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)

  start_time = datetime.datetime.now()
  # Pętla treningowa
  num_epochs = 50
  for epoch in range(num_epochs):
      net.train()
      for inputs, labels in train_loader:
          inputs, labels = inputs.to(device), labels.to(device)

          # Przepuszczenie danych przez sieć, obliczenie funkcji kosztu i wstecznej propagacji
          outputs = net(inputs)
          optimizer.zero_grad()
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
      print(f"epoch {epoch+1}/{num_epochs}, elapsed time: {datetime.datetime.now() - start_time}")


  # Testowanie modelu - obliczenie accuracy
  net.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for inputs, labels in test_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = correct / total
  return accuracy

Files already downloaded and verified
Files already downloaded and verified


In [71]:
train_and_eval(0.00032612430218809, 2, [943, 10], 32, 0.0652566450453942, 1.35021744016308e-05)
# Hyperparameters of trial number 84
#   Learning Rate: 0.0003261243021880896
#   Dropout Rate: 0.06525664504539422
#   Weight Decay: 1.3502174401630838e-05
#   Number of Layers: 2
#   Batch Size: 32
#   Layer Sizes: [943, 10]

epoch 1/50, elapsed time: 0:00:23.351481
epoch 2/50, elapsed time: 0:00:40.206933
epoch 3/50, elapsed time: 0:00:57.859807
epoch 4/50, elapsed time: 0:01:21.382910
epoch 5/50, elapsed time: 0:01:39.525503
epoch 6/50, elapsed time: 0:01:58.560174
epoch 7/50, elapsed time: 0:02:14.607538
epoch 8/50, elapsed time: 0:02:32.380440
epoch 9/50, elapsed time: 0:02:49.580538
epoch 10/50, elapsed time: 0:03:07.779247
epoch 11/50, elapsed time: 0:03:24.970432
epoch 12/50, elapsed time: 0:03:45.368555
epoch 13/50, elapsed time: 0:04:01.913200
epoch 14/50, elapsed time: 0:04:21.476160
epoch 15/50, elapsed time: 0:04:42.909519
epoch 16/50, elapsed time: 0:05:01.802934
epoch 17/50, elapsed time: 0:05:19.404777
epoch 18/50, elapsed time: 0:05:42.419396
epoch 19/50, elapsed time: 0:05:59.729635
epoch 20/50, elapsed time: 0:06:25.776633
epoch 21/50, elapsed time: 0:06:45.446591
epoch 22/50, elapsed time: 0:07:08.276091
epoch 23/50, elapsed time: 0:07:24.816539
epoch 24/50, elapsed time: 0:07:43.569060
e

0.6719

In [59]:
import optuna
import datetime
import logging
import sys

In [63]:
print(datetime.datetime.now())
def objective(trial):
    # Zakres przeszukiwania dla hiperparametrów
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    num_layers = int(trial.suggest_float("num_layers", 2.0, 10.0, step=1.0, log=False))
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])

    layer_sizes = []
    layer_sizes.append(int(trial.suggest_float("layer-1-size", 128, 1024, step=1.0, log=False)))
    for i in range(1, num_layers-1):
      name = "layer-{}-size".format(i+1)
      layer_sizes.append(int(trial.suggest_float(name, 64, 1024, step=1.0, log=False)))

    layer_sizes.append(10)

    print(f"Hyperparameters of trial number {trial.number}")
    print(f"  Learning Rate: {learning_rate}")
    print(f"  Dropout Rate: {dropout_rate}")
    print(f"  Weight Decay: {weight_decay}")
    print(f"  Number of Layers: {num_layers}")
    print(f"  Batch Size: {batch_size}")
    print(f"  Layer Sizes: {layer_sizes}")

    # Trening modelu z hiperparametrami
    accuracy = train_and_eval(
        learning_rate,
        num_layers,
        layer_sizes,
        batch_size,
        dropout_rate,
        weight_decay
    )
    print()
    print(f"  Accuracy: {accuracy}")
    print(f"  Elapsed time: {datetime.datetime.now() - trial.datetime_start}")
    print()
    print()
    print()

    return accuracy

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "mro-kadet-04"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, direction="maximize")

# Uruchomienie Optuna i konfiguracja badań
# study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

# Najlepsze konfiguracje hiperparametrów
print("Najlepsze hiperparametry: ", study.best_params)
print("Najlepsza dokładność: ", study.best_value)

2024-11-01 02:18:47.721311


[I 2024-11-01 02:18:49,694] A new study created in RDB with name: mro-kadet-04


A new study created in RDB with name: mro-kadet-04
A new study created in RDB with name: mro-kadet-04
A new study created in RDB with name: mro-kadet-04
A new study created in RDB with name: mro-kadet-04
A new study created in RDB with name: mro-kadet-04
A new study created in RDB with name: mro-kadet-04
A new study created in RDB with name: mro-kadet-04
Hyperparameters of trial number 0
  Learning Rate: 0.002037847496967966
  Dropout Rate: 0.4524419894039575
  Weight Decay: 0.0003643103187585856
  Number of Layers: 2
  Batch Size: 64
  Layer Sizes: [848, 10]


[I 2024-11-01 02:21:06,955] Trial 0 finished with value: 0.5717 and parameters: {'learning_rate': 0.002037847496967966, 'dropout_rate': 0.4524419894039575, 'weight_decay': 0.0003643103187585856, 'num_layers': 2.0, 'batch_size': 64, 'layer-1-size': 848.0}. Best is trial 0 with value: 0.5717.



  Accuracy: 0.5717
  Elapsed time: 0:02:17.149210



Trial 0 finished with value: 0.5717 and parameters: {'learning_rate': 0.002037847496967966, 'dropout_rate': 0.4524419894039575, 'weight_decay': 0.0003643103187585856, 'num_layers': 2.0, 'batch_size': 64, 'layer-1-size': 848.0}. Best is trial 0 with value: 0.5717.
Trial 0 finished with value: 0.5717 and parameters: {'learning_rate': 0.002037847496967966, 'dropout_rate': 0.4524419894039575, 'weight_decay': 0.0003643103187585856, 'num_layers': 2.0, 'batch_size': 64, 'layer-1-size': 848.0}. Best is trial 0 with value: 0.5717.
Trial 0 finished with value: 0.5717 and parameters: {'learning_rate': 0.002037847496967966, 'dropout_rate': 0.4524419894039575, 'weight_decay': 0.0003643103187585856, 'num_layers': 2.0, 'batch_size': 64, 'layer-1-size': 848.0}. Best is trial 0 with value: 0.5717.
Trial 0 finished with value: 0.5717 and parameters: {'learning_rate': 0.002037847496967966, 'dropout_rate': 0.4524419894039575, 'weight_decay': 0.00036431

...

[W 2024-11-01 19:05:06,224] Trial 241 failed with parameters: {'learning_rate': 0.0006216284066704859, 'dropout_rate': 0.08278753167707956, 'weight_decay': 3.5567834124304784e-05, 'num_layers': 2.0, 'batch_size': 32, 'layer-1-size': 968.0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\kadet\miniconda3\envs\MRO\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kadet\AppData\Local\Temp\ipykernel_2125232\3722706538.py", line 27, in objective
    accuracy = train_and_eval(
               ^^^^^^^^^^^^^^^
  File "C:\Users\kadet\AppData\Local\Temp\ipykernel_2125232\3817530194.py", line 29, in train_and_eval
    for inputs, labels in train_loader:
  File "c:\Users\kadet\miniconda3\envs\MRO\Lib\site-packages\torch\utils\data\dataloader.py", line 439, in __iter__
    return self._get_iterator()
           ^^^^^^^^^^^^^^^^^^^^
  File "

Trial 241 failed with parameters: {'learning_rate': 0.0006216284066704859, 'dropout_rate': 0.08278753167707956, 'weight_decay': 3.5567834124304784e-05, 'num_layers': 2.0, 'batch_size': 32, 'layer-1-size': 968.0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\kadet\miniconda3\envs\MRO\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kadet\AppData\Local\Temp\ipykernel_2125232\3722706538.py", line 27, in objective
    accuracy = train_and_eval(
               ^^^^^^^^^^^^^^^
  File "C:\Users\kadet\AppData\Local\Temp\ipykernel_2125232\3817530194.py", line 29, in train_and_eval
    for inputs, labels in train_loader:
  File "c:\Users\kadet\miniconda3\envs\MRO\Lib\site-packages\torch\utils\data\dataloader.py", line 439, in __iter__
    return self._get_iterator()
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kadet\miniconda3\en

[W 2024-11-01 19:05:06,233] Trial 241 failed with value None.


Trial 241 failed with value None.
Trial 241 failed with value None.
Trial 241 failed with value None.
Trial 241 failed with value None.
Trial 241 failed with value None.
Trial 241 failed with value None.
Trial 241 failed with value None.


KeyboardInterrupt: 